In [4]:
class Queue:
    def __init__(self):
        self.item=[]
    
    def queue_size(self):
        return len(self.item)
    
    def isEmpty(self):
        if len(self.item)>0:
            return True
        else:
            False
    
    def add_queue(self):
        
    
    def del_queue(self): #delete last=delete first come group 
        
        
    def postpone(self): #postpone 3 requests
        
    
    def cancel(self):  #delete certain one=cancel certain order

IndentationError: expected an indented block (<ipython-input-4-a35bad8f2150>, line 16)

In [7]:
# this class can be regarded as "class printer" in that case.
class Tables:
    def __init__(self, num, size):
        self.num=num # No. of the table
        self.size = size # Size of the table: for group of up to 2, 4 or 6.
        self.currentStatus = None # Is the table occupied or not.
        self.time_remaining = 0 
        
    def busy(self):
        if self.currentStatus != None:
            return True
        else:
            return False
    
    def tick(self):
        if self.currentStatus != None:
            self.time_remaining = self.time_remaining - 1
            if self.time_remaining <= 0:
                self.currentStatus = None
    
    def startNext(self, newtask):
        self.currentStatus = newtask
        self.time_remaining = newtask.time_needs() # here maybe generate a random time from 10 - 60 mins using generator

In [ ]:
# this class can be regarded as Task in that case
import random

class Groups:
    def __init__(self, time, size):
        self.timestamp = time # Time when group registered (entered into the queue)
        self.size = size #randomly define size from 1 - 6
    
    def get_stamp(self):
        return self.timestamp
    
    def get_size(self):
        return self.size
    
    def wait_time(self, current_time):
        return current_time - self.timestamp

In [2]:
def newCustomer():
    num = random.randrange(1, 61)
    if num == 60:
        return True
    else:
        return False


def simulation(number_tables_2, number_tables_4, number_tables_6, num_minutes):
    for i in range(number_tables_2):
        table_2_list.append(Tables(2))
    
    for i in range(number_tables_4):
        table_4_list.append(Tables(4))
    
    for i in range(number_tables_6):
        table_6_list.append(Tables(6))
        
    customer_queue = Queue()
    waiting_time = []
        
    for currentMinute in range(num_minutes):
        
        if newCustomer():
            customer = Customers(currentMinute)
            customer_queue.enqueue(customer)
            
        if Customers.size == 1 or Customers.size == 2:
            for i in range(number_tables_2):
                if (not number_tables_2[i].busy()) and (not customer_queue.isEmpty()):
                    nexttask = customer_queue.dequeue()
                    waiting_time.append(nexttask.wait_time(currentMinute))
                    number_tables_2[i].startNext(nexttask)
                    break
            number_tables_2[i].tick

        elif Customers.size == 3 or Customers.size == 4:
            for i in range(number_tables_4):
                if (not number_tables_4[i].busy()) and (not customer_queue.isEmpty()):
                    nexttask = customer_queue.dequeue()
                    waiting_time.append(nexttask.wait_time(currentMinute))
                    number_tables_4[i].startNext(nexttask)
                    break
            number_tables_4[i].tick
            
        else:
            for i in range(number_tables_6):
                if (not number_tables_6[i].busy()) and (not customer_queue.isEmpty()):
                    nexttask = customer_queue.dequeue()
                    waiting_time.append(nexttask.wait_time(currentMinute))
                    number_tables_6[i].startNext(nexttask)
                    break
            number_tables_6[i].tick
    
        averageWait = sum(waiting_time)/len(waiting_time)
        print('Average wait',averageWait,' minutes for party of ', Customers.size, ',and',
        customer_queue.size(), 'customers remaining.')

In [8]:
t1 = Tables(2,2)